In [1]:
import pandas
import pandas as pd
import numpy as np
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import matplotlib.pyplot as plt 
import datetime as dt
from datetime import datetime

In [2]:
def import_binance_data(folder_name):
    schema = None
    partitioning = ds.partitioning(schema, flavor="hive")
    dataset = pq.ParquetDataset(
        # Replace URI with path.
        folder_name,
        #filesystem=filesystem,
        #filters=filters,
        partitioning=partitioning,
        use_legacy_dataset=False,
    )
    columns = None
    table = dataset.read_pandas(columns=columns)
    binance_data = table.to_pandas()
    return binance_data

In [6]:
def clean_binance_data(binance_df):
    #drop the exchange_id category since all the rows say 'binance' and we can just acknowedge that fact somewhere else
    binance_data = binance_df.drop(['year','month','timestamp','exchange_id'],axis = 1)
    #need to reset the index because the df currently has datetime timestamp object as indexing
    binance_data = binance_data.reset_index()
    #creating a category for the following: DD-MM-YY HH:MM, this makes it easier to group all the minutes and take their average
    binance_data['timestamp'] = binance_data['timestamp'].dt.strftime("%D %H:%M")
    #groups all the minutes together by bitcoin type
    binance_data = binance_data.groupby(['timestamp','currency_pair']).mean()
    binance_data = binance_data.reset_index()
    return binance_data

In [7]:
local_folder_name = "C:/Users/Dylan Celli/Downloads/bulk.airflow.downloaded_1min.parquet.ohlcv.futures.v7.ccxt.binance.v1_0_0"
#assumes that the data is already unzipped
binance_df = import_binance_data(local_folder_name)
binance_data = clean_binance_data(binance_df)
#need the index = False bc otherwise it adds an unecessary column for the index as data
binance_data.to_csv('clean_binance.csv',index = False)